In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import f1_score

In [2]:
catboost_oof = pd.read_csv("../input/ykc2-cb-ngt/catboost_text_vector_oof.csv")
catboost_test = pd.read_csv("../input/ykc2-cb-ngt/catboost_text_vector_test.csv")
nn_oof = pd.read_csv("../input/ykc2-catboost-text-nn/catboost_text_nn_oof.csv")
nn_test = pd.read_csv("../input/ykc2-catboost-text-nn/catboost_text_nn_test.csv")
lgb_oof = pd.read_csv("../input/ykc2-catboost-ngt-top-n/catboost_text_vector_oof.csv")
lgb_test = pd.read_csv("../input/ykc2-catboost-ngt-top-n/catboost_text_vector_test.csv")
train = pd.read_csv("../input/ykc-2nd/train.csv")
sub = pd.read_csv("../input/ykc-2nd/sample_submission.csv")
target = 'department_id'

In [3]:
target_df = train[['department_id']]

In [4]:
#normalizing nn
nn_oof = nn_oof.div(nn_oof.sum(axis=1), axis=0)
nn_test = nn_test.div(nn_test.sum(axis=1), axis=0)

In [5]:
#find the best coeeficient
scores = []
for i in range(11):
    for j in range(11-i):
        cb_coef = i/10
        nn_coef = j/10
        lgb_coef = 1-cb_coef-nn_coef
        train_df = target_df.join(cb_coef * catboost_oof + nn_coef * nn_oof + lgb_coef * lgb_oof)

        y_preds = train_df.drop(target, axis=1).values
        y = train_df[target]

        f1 = f1_score(y, np.argmax(y_preds, axis = 1), average = "micro")
        scores.append({'cb_coef':cb_coef, 'nn_coef':nn_coef, 'lgb_coef':lgb_coef, 'f1':f1})
score_df = pd.DataFrame(scores)
score_df.sort_values('f1', ascending=False).head()

,cb_coef,nn_coef,lgb_coef,f1
53,0.6,0.2,2.000000e-01,0.812535
54,0.6,0.3,1.000000e-01,0.812334
59,0.7,0.3,5.551115e-17,0.812173
58,0.7,0.2,1.000000e-01,0.812173
49,0.5,0.4,1.000000e-01,0.811408


In [6]:
idx = score_df.f1.argmax()
cb_coef = score_df.loc[idx, 'cb_coef']
nn_coef = score_df.loc[idx, 'nn_coef']
lgb_coef = score_df.loc[idx, 'lgb_coef']
y_pred = cb_coef * catboost_test + nn_coef * nn_test + lgb_coef * lgb_test
pred_final = y_pred.values

In [7]:
pred_final_value = np.argmax(np.array(pred_final), axis=1)
sub["department_id"] = pred_final_value
sub.to_csv("submission.csv", index = False)
sub.head()

,product_id,department_id
0,24842,18
1,24843,6
2,24844,6
3,24845,6
4,24846,2
